In [1]:
######## YCrCb
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while(1):

    # Take each frame
    _, frame = cap.read()

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB)

    # define range of skin color in HSV
    lower_skin = np.array([0,138,67])
    upper_skin = np.array([255,173,133])
    
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)
    
    kernel=np.ones((3,3),np.uint8)
    erosion=cv2.erode(mask,kernel,iterations=1)
    dilation=cv2.dilate(mask,kernel,iterations=1)
    
    smoothed=cv2.filter2D(dilation,-1,kernel)
    
    blur=cv2.GaussianBlur(dilation,(15,15),0)
    median=cv2.medianBlur(dilation,15)
    bilateral=cv2.bilateralFilter(dilation,15,75,75)
    
    
   
   #dst = cv2.fastNlMeansDenoisingColored(res,None,10,10,7,21)
    
    cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    
    cv2.imshow('erosion',erosion)
    cv2.imshow('dilation',dilation)
    
    cv2.imshow('smoothed',smoothed)
    cv2.imshow('blur',blur)
    cv2.imshow('bil',bilateral)

    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
#press ESC to exit
cv2.destroyAllWindows()

error: OpenCV(3.4.2) c:\miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<3,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'
